In [1]:
import numpy as np

In [2]:
def soft_threshold(rho: float, lam: float) -> float:
    """Soft-thresholding operator S(rho, lam)."""
    if rho > lam:
        return rho - lam
    if rho < -lam:
        return rho + lam
    return 0.0

In [35]:
def lasso_coordinate_descent(
    X: np.ndarray,
    y: np.ndarray,
    lam: float,
    fit_intercept: bool = True,
    max_iter: int = 10_000,
    tol: float = 1e-6,
    verbose: bool = False
):
    """
    Solve Lasso via cyclic (or shuffled) coordinate descent:

        minimize_w,b   (1/(2n)) ||y - (Xw + b)||_2^2 + lam * ||w||_1

    """
    X = np.asarray(X, dtype=float)
    y = np.asarray(y, dtype=float).ravel()
    n, p = X.shape

    rng = np.random.default_rng(100)

    # --- Center / standardize ---
    x_mean = np.zeros(p)
    x_scale = np.ones(p)
    y_mean = 0.0

    Xs = X
    ys = y

    y_mean = ys.mean()
    ys = ys - y_mean

    # Precompute column norms: (1/n) * sum x_ij^2
    col_norm = (Xs * Xs).mean(axis=0)

    # --- Initialize ---
    w = np.zeros(p)
#     w[[1,5,6,7,8]] = [1,2,1,1,0.5]
    b = 0.0  # in centered space, intercept is 0; we reconstruct later.

    # Residual r = y - Xw  (in centered/standardized space, and y centered if intercept)
    r = ys.copy()

    # Coordinate descent
    for it in range(max_iter):
        w_old = w.copy()

        # Optionally shuffle feature order each epoch
        order = np.arange(p)
        
        rng.shuffle(order)

        for j in order:
            if col_norm[j] == 0.0:
                # Feature is all zeros after preprocessing
                continue

            # Add back current contribution of feature j
            r += Xs[:, j] * w[j]

            # rho = (1/n) * x_j^T r
            rho = (Xs[:, j] @ r) / n

            # Closed-form update (note: objective uses (1/(2n)) so threshold is lam)
            w[j] = soft_threshold(rho, lam) / col_norm[j]

            # Remove updated contribution
            r -= Xs[:, j] * w[j]

        # Convergence check (in coefficient space)
        max_change = np.max(np.abs(w - w_old))

        if max_change < tol:
            break

    # --- Unscale to original feature space ---
    w_orig = w / x_scale

    # Reconstruct intercept in original space:
    # y ≈ X w_orig + intercept
    
    intercept = y_mean - x_mean @ w_orig

    return {
        "coef": w_orig,
        "intercept": intercept,
        "n_iter": it + 1,
        "converged": (max_change < tol),
    }

In [36]:
# ------------------- Example usage -------------------
rng = np.random.default_rng(0)
n, p = 1000, 200
X = rng.normal(size=(n, p))
true_w = np.zeros(p)
true_w[[1, 2, 5, 9]] = [11.0, -9, -10, 20.0]
sigma_temp = 0.1
y = X @ true_w + rng.normal(scale=sigma_temp, size=n)

# print(y)

In [37]:
# lam_0 = sigma_temp*np.sqrt(np.log(p)/n)
lam_0 = 0.1
sol = lasso_coordinate_descent(X, y, lam=lam_0, fit_intercept=True, verbose=True)

In [38]:
print("coef (first 10):", np.round(sol["coef"][:10],2))

coef (first 10): [ 0.   10.9  -8.91  0.    0.   -9.91  0.    0.    0.   19.9 ]


In [33]:
print("n_iter:", sol["n_iter"], "converged:", sol["converged"])

n_iter: 8 converged: True


In [34]:
print(np.sum(np.abs(sol["coef"][11:])))

0.0
